In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from sklearn import svm

In [ ]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device):
    """ Training a model for one epoch """
    
    loss_list = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list


@torch.no_grad()
def eval_model(model, eval_loader, criterion, device):
    """ Evaluating the model for either validation or test """
    correct = 0
    total = 0
    loss_list = []
    for images, labels in eval_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss


def train_model(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs):
    """ Training a model for a given number of epochs"""
    
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
           
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = eval_model(
                    model=model, eval_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train_epoch(
                model=model, train_loader=train_loader, optimizer=optimizer,
                criterion=criterion, epoch=epoch, device=device
            )
        scheduler.step()
        train_loss.append(mean_loss)
        loss_iters = loss_iters + cur_loss_iters
        
        if(epoch % 5 == 0 or epoch==num_epochs-1):
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"    Train loss: {round(mean_loss, 5)}")
            print(f"    Valid loss: {round(loss, 5)}")
            print(f"    Accuracy: {accuracy}%")
            print("\n")
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc

In [ ]:
def lock_layers(model,params_dict):
    """locks all convolutional layers except the classifier one before training"""
    
    model_params = params_dict.keys()  
    for key in model_params:
        if key.startswith('classifier'):
            params_dict[key].requires_grad = True
        else:  
            params_dict[key].requires_grad = False

In [ ]:
def model_train_test_plot(model,
                          title,
                          train_loader,
                          valid_loader,
                          test_loader,
                          optimizer,
                          scheduler,
                          criterion = nn.CrossEntropyLoss(),
                          num_epochs = 15,
                          flag_fix_layers = False):
    """
    Trains, tests and analysis a model
    Params:
    ----------
    title: str
        Title of the model
    flag_fix_layers: Bool 
        Locks classification layers in the model if True
    
    Return
    ----------
    accuracy value and plots of the model
    """
    model = model.to(device)
    
    if flag_fix_layers: # parameters in the feature extraction (convolution) layers are frozen during training
        params_dict = model.state_dict() 
        lock_layers(model,params_dict) 
    
    # train model
    train_loss, val_loss, loss_iters, valid_acc = train_model(
            model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion,
            train_loader=train_loader, valid_loader=valid_loader, num_epochs=15
    )
    # plot model analysis
    plot_analysis(loss_iters,train_loss, val_loss,valid_acc,title) #fixed_models_titles[idx]
    # Test on testset
    print(f'Currently testing: {title}...\n')
    accuracy, _ = eval_model(model, test_loader, criterion, device)
    print(f"Accuracy of model {title} on the testset: {round(np.max(accuracy),2)}%\n")
    
    return accuracy
    